In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options 
from selenium.webdriver.common.by import By
import time
import xlsxwriter
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
# urls of the youtube channels
urls = [
    # 'https://www.youtube.com/@_SMWX/videos'
    # 'https://www.youtube.com/@konvo_za/videos'
    # 'https://www.youtube.com/@kingdavidstudio1/videos'
    'https://www.youtube.com/@TheHustlersCornerSA/videos'
    # 'https://www.youtube.com/@MajitaMonday/videos'
    # 'https://www.youtube.com/@justifyworldwide/videos'
    #'https://www.youtube.com/@culturespotlight/videos'
    # 'https://www.youtube.com/@podcastandchillnetwork/videos'
    ]

In [12]:
# specify the path to the Edge WebDriver
service = Service(executable_path='C:/Users/vince/Downloads/edgedriver_win64/msedgedriver.exe')

# set up Edge options (optional, can be customized as needed)
options = Options()
options.add_argument("start-maximized")  # Example option to open Edge maximized

# initialize the Edge WebDriver with the service and options
driver = webdriver.Edge(service=service, options=options)

In [13]:
# initialize an empty list of videos
video_list = []

# iterate through the channels 
for url in urls:
    driver.get('{}/videos?view=0&sort=p&flow=grid'.format(url))
    
    # max number of scroll attempts
    max_scroll_attempts = 70
    scroll_attempts = 0
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # scroll to the bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(10)  # Wait for content to load
        
        # get the new page height after scrolling
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # if the height is the same, it means no new content is loading
        if new_height == last_height or scroll_attempts >= max_scroll_attempts:
            break
        
        last_height = new_height  # update the height for the next iteration
        scroll_attempts += 1  # increment the scroll attempts

    try:
        # find all video elements on the page
        videos = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-rich-grid-media')
        duration_elements = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-rich-grid-renderer')
        
        for video in videos:
            title = video.find_element(By.XPATH, './/*[@id="video-title"]').text
            views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text.replace('views', '')
            #when = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text

            # incase of live streaming : updated XPath for 'posted' (when the video was uploaded)
            try:
                when = WebDriverWait(video, 10).until(
                    EC.presence_of_element_located((By.XPATH, './/*[@id="metadata-line"]/span[2]'))
                ).text
            except Exception as e:
                when = "N/A" 
            
            video_item = {
                'title': title,
                'views': views,
                'posted': when
            }
            
            video_list.append(video_item) 
    
    except Exception as e:
        print(f"Error processing {url}: {e}")

# closing the driver
driver.quit()


In [14]:
# check if the video_list is empty or loaded

if not video_list:
    print('video_list is empty')
else :
    print(video_list)

[{'title': 'DR KHEHLELEZI - 2025 PREDICTIONS | Isintu, Ubunyanga, Ukufa, Umuthi, Ukuthwala, Ubuthakathi, Isgubhu', 'views': '1.4K ', 'posted': '2 hours ago'}, {'title': 'BOITUMELO MONAGENG - HOW I FOUNDED SWYPA & TEMBISA NEWS | Uber Eats, Google, Facebook, Takealot,Mr D', 'views': '3.7K ', 'posted': '7 days ago'}, {'title': '“I WAS DISCOVERED BY CHESTER” - INNOCENT “BOBO” MASUKU | Acting, Yizo Yizo, Popeye & Spinach, Zola', 'views': '154K ', 'posted': '2 weeks ago'}, {'title': 'THULANI MTSWENI | Mr Easy Loo, Rhythm City, Gomora, Isibaya, Isidingo, Isono, Mabaso Family Reunion', 'views': '318K ', 'posted': '3 weeks ago'}, {'title': 'TK NCIZA | BIRTHDAY, ANC, MUSIC, BUSINESS, TS RECORDS, ZAHARA, SJAVA, SKEEM GP, ARTHUR, DR KHUMALO', 'views': '13K ', 'posted': '4 weeks ago'}, {'title': 'PRINCE MASHELE - 2024 POLITICAL ROUND UP | ANC, DA, MK, EFF, PA, IFP, ISRAEL, US, BOTS, ZIM, LESOTHO', 'views': '171K ', 'posted': '1 month ago'}, {'title': 'BONGINKOSI KHANYILE | MK Turns 1, Unity, Progre

In [15]:
# create a dataframe
df_oc = pd.DataFrame(video_list)

# display the data frame
print(df_oc)

                                                 title  views       posted
0    DR KHEHLELEZI - 2025 PREDICTIONS | Isintu, Ubu...  1.4K   2 hours ago
1    BOITUMELO MONAGENG - HOW I FOUNDED SWYPA & TEM...  3.7K    7 days ago
2    “I WAS DISCOVERED BY CHESTER” - INNOCENT “BOBO...  154K   2 weeks ago
3    THULANI MTSWENI | Mr Easy Loo, Rhythm City, Go...  318K   3 weeks ago
4    TK NCIZA | BIRTHDAY, ANC, MUSIC, BUSINESS, TS ...   13K   4 weeks ago
..                                                 ...    ...          ...
555  Episode 5 | Zakes Bantwini | Black Empowerment...   32K   5 years ago
556  Episode 4 | Busi Mkhumbuzi Pooe | Activism | U...  7.5K   5 years ago
557  Episode 3 | Michelle Mosalakae | Acting career...  7.4K   5 years ago
558  Episose 2 | Dr Nokukhanya Khanyile | Medicine ...   40K   5 years ago
559  Episode 1| Jabulani 'Cashflow' Ngcobo | 'Cashf...  197K   5 years ago

[560 rows x 3 columns]


In [16]:
# Function to help convert view counts

def convert_views_count(view):

    try:
        float_view = float(view)
        return float_view 
    except ValueError:
        view = view.strip()

        if 'K' in view:
            return float(view.replace('K', '')) * 1000
        elif 'M' in view:
            return float(view.replace('M', '')) * 1000000
        else:
            return float(view)
    
# Apply the convert_views_count function to the 'views' column
df_oc['views'] = df_oc['views'].apply(convert_views_count)

# removing the word ago in the columnd posted
df_oc['posted'] = df_oc['posted'].str.replace('ago','')

In [17]:
print(df_oc)

                                                 title     views    posted
0    DR KHEHLELEZI - 2025 PREDICTIONS | Isintu, Ubu...    1400.0  2 hours 
1    BOITUMELO MONAGENG - HOW I FOUNDED SWYPA & TEM...    3700.0   7 days 
2    “I WAS DISCOVERED BY CHESTER” - INNOCENT “BOBO...  154000.0  2 weeks 
3    THULANI MTSWENI | Mr Easy Loo, Rhythm City, Go...  318000.0  3 weeks 
4    TK NCIZA | BIRTHDAY, ANC, MUSIC, BUSINESS, TS ...   13000.0  4 weeks 
..                                                 ...       ...       ...
555  Episode 5 | Zakes Bantwini | Black Empowerment...   32000.0  5 years 
556  Episode 4 | Busi Mkhumbuzi Pooe | Activism | U...    7500.0  5 years 
557  Episode 3 | Michelle Mosalakae | Acting career...    7400.0  5 years 
558  Episose 2 | Dr Nokukhanya Khanyile | Medicine ...   40000.0  5 years 
559  Episode 1| Jabulani 'Cashflow' Ngcobo | 'Cashf...  197000.0  5 years 

[560 rows x 3 columns]


In [18]:
try:
    df_oc.to_csv('Output Files/TheHustlersCornerSA_Podcast.csv', index = False )
    print("Successfully exported to csv file")

except Exception as e:
    print(e)

Successfully exported to csv file


In [19]:
try:
    df_oc.to_excel('Output Files/TheHustlersCornerSA_Podcast.xlsx', index=False)
    print("Successfully exported to xlsx file")

except Exception as e:
    print(e)

Successfully exported to xlsx file
